# Using `surprise`

See the documentation [here](https://surprise.readthedocs.io/en/stable/getting_started.html)!

In [2]:
import surprise
from surprise.prediction_algorithms import *
import pandas as pd
import numpy as np
import datetime as dt

## Agenda

SWBAT:

- use the `surprise` package to build recommendation engines.

In [3]:
data = surprise.Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /Users/jacobserfaty/.surprise_data/ml-100k


Now that we've downloaded the data, we can find it in a hidden directory:

In [4]:
df = pd.read_csv('~/.surprise_data/ml-100k/ml-100k/u.data',
            sep='\t', header=None)
df = df.rename(columns={0: 'user', 1: 'item', 2: 'rating', 3: 'timestamp'})
df

,user,item,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


## Data Exploration

In [5]:
df['user'].nunique()

943

In [6]:
df['item'].nunique()

1682

In [7]:
stats = df[['rating', 'timestamp']].describe()
stats

,rating,timestamp
count,100000.000000,1.000000e+05
mean,3.529860,8.835289e+08
std,1.125674,5.343856e+06
min,1.000000,8.747247e+08
25%,3.000000,8.794487e+08
50%,4.000000,8.828269e+08
75%,4.000000,8.882600e+08
max,5.000000,8.932866e+08


In [8]:
print(dt.datetime.fromtimestamp(stats.loc['min', 'timestamp']))
print(dt.datetime.fromtimestamp(stats.loc['max', 'timestamp']))

1997-09-19 23:05:10
1998-04-22 19:10:38


In [9]:
read = surprise.Reader('ml-100k')

In [10]:
read.rating_scale

(1, 5)

## Modeling

In [11]:
train, test = surprise.model_selection.train_test_split(data, random_state=42)

In [12]:
model = KNNBasic().fit(train)

Computing the msd similarity matrix...
Done computing similarity matrix.


$\hat{r}_{ui} = \frac{
    \sum\limits_{v \in N^k_i(u)} \text{sim}(u, v) \cdot r_{vi}}
    {\sum\limits_{v \in N^k_i(u)} \text{sim}(u, v)}$
    OR
$\hat{r}_{ui} = \frac{
    \sum\limits_{j \in N^k_u(i)} \text{sim}(i, j) \cdot r_{uj}}
    {\sum\limits_{j \in N^k_u(i)} \text{sim}(i, j)}$

In [13]:
model2 = SVD().fit(train)

$\sum_{r_{ui} \in R_{train}} \left(r_{ui} - \hat{r}_{ui} \right)^2 +
    \lambda\left(b_i^2 + b_u^2 + ||q_i||^2 + ||p_u||^2\right)$

In [14]:
model3 = NMF().fit(train)

$\hat{r}_{ui} = q_i^Tp_u$

In [15]:
#most similiar to movie id 51 using from KNNBasic
model.get_neighbors(iid=51, k=1)

[65]

In [ ]:
conds = [df['item'] == 51, df['item'] == 65]
choices = 2*[True]

df.loc[np.select(conds, choices, default=False)].sort_values('user')

## Evaluation

In [ ]:
# get predictions
model.test(test)

In [ ]:
#KNNBasic
surprise.accuracy.mae(model.test(test))

In [ ]:
#KNNBasic
surprise.accuracy.rmse(model.test(test))

In [ ]:
#SVD
surprise.accuracy.mae(model2.test(test))

In [ ]:
#SVD
surprise.accuracy.rmse(model2.test(test))

In [ ]:
#NMF
surprise.accuracy.mae(model3.test(test))

In [ ]:
#NMF
surprise.accuracy.rmse(model3.test(test))